# Упражнение 2. Качество данных

## Utils

In [6]:
import pandas as pd
import utils.scd as scd
from datetime import date, datetime, timedelta
import warnings

warnings.filterwarnings('ignore', message="registration of accessor <class '__main__.CustomAccessor'>")
warnings.filterwarnings('ignore', message="registration of accessor <class '__main__.SCD'>")
warnings.simplefilter(action='ignore', category=FutureWarning)

## Задание 2.0

В упражнении 1 уже скачивали данные об учебных планах. Убедитесь в том, что эти данные у вас сохранились.

In [1]:
%ls data

academic_plan_id.csv
academic_plan_id.csv.dvc
academic_plans.csv
academic_plans.csv.dvc
academic_plans_in_field_of_study.csv
academic_plans_in_field_of_study.csv.dvc
all_details.json
all_details.json.dvc
editors.csv
editors.csv.dvc
fields_of_study.csv
fields_of_study.csv.dvc
structural_units.csv
structural_units.csv.dvc


## Задание 2.1 Получение данных

In [4]:
from utils.data_collector import OPITMODataCollector

data_collector = OPITMODataCollector()

Идентификаторы всех учебных планов, имеющихся в op.itmo.ru

In [3]:
# Get new ids from API:
# ids = data_collector.get_academic_plans_ids()
# ids = list(ids)

# User stored ids in DVC:
ids = data_collector.get_data('data/academic_plan_id')['academic_plan_id'].tolist()

Found 60 pages

In [7]:
df_aps = pd.DataFrame(ids, columns=['academic_plan_id'])
df_aps.custom.to_csv_with_time('academic_plan_id')
df_aps

,academic_plan_id
0,6796
1,6797
2,6798
3,6799
4,6800
...,...
592,7501
593,7502
594,7503
595,7504


In [8]:
# Save collected data in DVC
# data_collector.save_data(df_aps, 'academic_plan_id')

# Or Local
# df_aps.to_csv(f'data/ids_{time.time()}.csv')

По этим планам получить детальное описание

In [10]:
# Update data via API:
all_details = data_collector.get_academic_plan_details(list(ids))

# User stored data in DVC:
# all_details = data_collector.get_data('data/all_details', file_type='json')

                ||||||||||||||||||||||||||||||||||||||||||||||||||
                ||||||||||||||||||||||||||||||||||||||||||||||||||
                ||||||||||||||||||||||||||||||||||||||||||||||||||
                ||||||||||||||||||||||||||||||||||||||||||||||||||
                ||||||||||||||||||||||||||||||||||||||||||||||||||
                ||||||||||||||||||||||||||||||||||||||||||||||||||
                ||||||||||||||||||||||||||||||||||||||||||||||||||
                ||||||||||||||||||||||||||||||||||||||||||||||||||
                ||||||||||||||||||||||||||||||||||||||||||||||||||
                ||||||||||||||||||||||||||||||||||||||||||||||||||
                ||||||||||||||||||||||||||||||||||||||||||||||||||
                |||||||||||||||||||||||||||||||||||||||||||||||

In [7]:
from utils.json_dict_converter import JsonDictConverter

In [8]:
# Save collected data in DVC
# data_collector.save_data(all_details, 'all_details')

# Or Local
JsonDictConverter().to_json(all_details, json_name='data/all_details', timed=True)

Медленно меняющиеся измерения (от англ. Slowly Changing Dimensions, SCD) — механизм отслеживания изменений в данных измерения. [Выделяют несколько типов SCD2. ](https://ru.wikipedia.org/wiki/%D0%9C%D0%B5%D0%B4%D0%BB%D0%B5%D0%BD%D0%BD%D0%BE_%D0%BC%D0%B5%D0%BD%D1%8F%D1%8E%D1%89%D0%B5%D0%B5%D1%81%D1%8F_%D0%B8%D0%B7%D0%BC%D0%B5%D1%80%D0%B5%D0%BD%D0%B8%D0%B5).

Реализуйте механизм SCD2. Добавилась строка и столбцы с датой вступления изменения в силу (столбце с TRUE/FALSE необязательно)



Мы решили добавлять колонку со статусом, чтобы отслеживать **полностью удаленные** значения.

Получаем новые данные

In [22]:
from utils.data_extractor import DataExtractor

data = JsonDictConverter().from_json("data/all_details_24_10_2023")

new_data = DataExtractor(data)

Получаем старые данные

In [23]:
data = JsonDictConverter().from_json("data/all_details")

old_data = DataExtractor(data)

Старым данным добавляем функционал scd

In [24]:
for df in old_data.dfs:
    old_data.dfs[df].scd.add_effective_date(current_date=datetime.now()-timedelta(14), inplace=True)
    old_data.dfs[df].scd.add_status(inplace=True)

К старым данным добавляем новые данные

In [25]:
for df in old_data.dfs:
    old_data.dfs[df] = old_data.dfs[df].scd.update(new_data.dfs[df])

Создаем локальные копии, которые в дальнейшем будем использовать для измерения качества данных

In [26]:
old_data.create_local_copies()

## Задание 2.2 Качество данных

Необходимо выбрать не менее трех характеристик качества данных и сделать профилирование данных, которое позволит измерить эти характеристики. Например, полнота и визуализация, на которой видны пропущенные значения.

В 1996 году Ричард Уанг и Дайана Стронг предложили модель оценки качества данных по следующим 15 параметрам (измерениям) их восприятия потребителями
* Качество данных как таковых:
1. точность;
2. объективность;
3. убедительность;
4. репутация источника.
* Контекстуальное качество данных:
1. полезность;
2. релевантность;
3. актуальность;
4. полнота;
5. достаточность объема данных.
* Репрезентативность:
1. интерпретируемость;
2. понятность;
3. логичность представления;
4. лаконичность представления.
* Наличие подтверждения качества данных:
1. доступность;
2. безопасность доступа и защита от несанкционированного доступа.

В этой работе рассмотрим следующие:
1. точность (accuracy),
2. завершенность (completeness),
3.

In [29]:
old_data.dfs["editors"]

,id,username,first_name,last_name,email,isu_number,effective_date,status
15,100.0,165829,Дарья,Плужникова,,165829,10_10_2023,True
13,1004.0,165700,Екатерина,Колесник,,165700,10_10_2023,True
60,102.0,145740,Алена,Иванова,aeivanova@itmo.ru,145740,10_10_2023,True
30,1033.0,307875,Дарина,Клименко,,307875,10_10_2023,True
94,1034.0,166046,Елена,Сучкова,,166046,10_10_2023,True
...,...,...,...,...,...,...,...,...
101,942.0,223648,Оксана,Солдатова,ovsoldatova@itmo.ru,223648,10_10_2023,True
22,946.0,338220,Илана,Розман,,338220,10_10_2023,True
11,97.0,276726,Анна,Карпова,aukarpova@itmo.ru,276726,10_10_2023,True
29,971.0,104397,Анна,Орлова,,104397,10_10_2023,True


## Задание 2.3 Метаданные

Подготовьте наиболее полные метаданные по данным, полученным в задании 2.1

# Критерии и баллы

Распределение исходя из 30 баллов.



*   2.0 - 1 балл
*   2.1 - 8 баллов
*   2.2 - 12 баллов
*   2.3 - 9 баллов

